In [ ]:
from langsmith import Client

client = Client()

In [ ]:
import pandas as pd

test_data = pd.read_csv('../../rzd_data/qna.csv')

In [ ]:
import random
from typing import Dict, List


def create_examples(df) -> List[Dict[str, str]]:
    examples = []
    for i, row in df.iterrows():
        examples.append({
            "question": row['Вопрос'],
            "answer": row['Ответ'],
        })

    return examples

In [ ]:
dataset_name = "rzd_22"
dataset = client.read_dataset(dataset_name=dataset_name)

In [ ]:
examples = create_examples(test_data)

inputs, outputs = zip(
    *[({"question": row["question"]}, row) for row in examples]
)

In [ ]:
client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

In [ ]:
import requests


def predict(_inputs: dict) -> dict:
    response = requests.post('http://0.0.0.0:8080/predict', json=_inputs)
    return response.json()

In [ ]:
from pydantic import BaseModel, Field


class EvalResponse(BaseModel):
    reason: str = Field(..., description="Причина выставления оценки")
    score: int = Field(..., description="Оценка ответа поданного на проверку")

In [ ]:
import httpx
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    model="gpt-4o-mini",
)

In [ ]:
eval_system = """\
Вы учитель, который проверяет викторину.

Вам будут предоставлены ВОПРОС, ПРАВИЛЬНЫЙ ОТВЕТ и ОТВЕТ УЧЕНИКА.

Вот критерии оценки:

(1) Оценивайте ответы учащихся ТОЛЬКО на основе их фактической точности относительно правильного ответа.

(2) Убедитесь, что ответ ученика не содержит противоречивых утверждений.

(3) Допустимо, если ответ ученика содержит больше информации, чем правильный ответ, при условии, что она фактически точна относительно правильного ответа.

Оценка:

Оценка 1 означает, что ответ ученика соответствует всем критериям. Это самая высокая (лучшая) оценка.

Оценка 0 означает, что ответ ученика не соответствует всем критериям. Это самая низкая оценка, которую вы можете поставить.

Объясните свое рассуждение пошагово, чтобы убедиться, что ваши выводы корректны.

Избегайте просто указывать правильный ответ в самом начале."""

eval_human = """\
QUESTION: {question}
GROUND TRUTH ANSWER: {correct_answer}
STUDENT ANSWER: {student_answer}"""

In [ ]:
eval_prompt = ChatPromptTemplate.from_messages([
    ('system', eval_system),
    ('human', eval_human),
])

In [ ]:
def exact_match_class_1(run, example):
    return {"score": run.outputs["class_1"] == example.outputs["class_1"]}


def exact_match_class_2(run, example):
    return {"score": run.outputs["class_2"] == example.outputs["class_2"]}


def exact_match_bz_answer(run, example):
    return {"score": run.outputs["answer_from_bz"] == example.outputs["answer_from_bz"]}


def check_bz_answer_in_docs(run, example):
    answer_from_bz = example.outputs["answer_from_bz"]
    return {"score": answer_from_bz in run.outputs["docs"]}


def middle_num_docs(run, example):
    return {"score": run.outputs["total_docs"]}


def check_answer_correctness(run, example):
    model_with_so = model.with_structured_output(EvalResponse)
    evaluate_chain = eval_prompt | model_with_so
    result = evaluate_chain.invoke({
        "student_answer": run.outputs["answer"],
        "correct_answer": example.outputs["answer"],
        "question": example.inputs["question"],
    })
    return {"score": result.score, "description": result.reason, "key": "answer_score"}

In [ ]:
from langsmith import evaluate

evaluate(
    predict,
    data=dataset_name,
    evaluators=[check_answer_correctness],
    metadata={"revision_id": "v.0.1.0"},
)